## Interactive Polarization Ellipse

This notebook plots the ellipse drawn by the two-dimensional electric field vector of a monochromatic electromagnetic wave, demonstrating the relation between the real-valued Cartesian coordinates of the electric field and the complex-valued analytic representation of those coordinates (as a function of time).  Use the slider bars to adjust the amplitude, orientation and ellipticity of the polarization ellipse.  

The following block of code creates the objects (lines and arrows) that will be used to plot the ellipses, configures the plot axes, and creates the slider bars.

In [1]:
from numpy import linspace, exp, cos, sin, pi, real
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from ipywidgets import interactive_output, FloatSlider, HBox, VBox, Layout
from IPython.display import display
        
# Set up the figure and the objects to hold plot data
fig = plt.figure(figsize=(8,8))
    
real_ellipse, = plt.plot([], [], color='k', linewidth=2, label=r"Equation 4")
comp_ellipse, = plt.plot([], [], color='r', linestyle='dotted', linewidth=2, label=r"Equation 5")

hw, hl = 0.06, 0.10
real_arrow = plt.arrow(0,0,1,1, head_width=hw, head_length=hl, fc='black', ec='black')
comp_arrow = plt.arrow(0,0,1,1, head_width=hw, head_length=hl, fc='red', ec='red')

# Set up the figure axes, etc.
plt.xlim(-1.0, 1.0)
plt.ylim(-1.0, 1.0)
plt.legend(loc="upper left")
    
# Place the figure spines at the origin
ax = plt.gca()
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
ax.xaxis.set_ticks_position('bottom')
ax.spines['bottom'].set_position(('data',0))
ax.yaxis.set_ticks_position('left')
ax.spines['left'].set_position(('data',0))

# Turn off tick labels
ax.set_yticklabels([])
ax.set_xticklabels([])
    
plt.close()

# Create slider bars
r_val   = FloatSlider(min=0, max=1.0, step=0.1, value=1.0, description="$r$")
chi_deg = FloatSlider(min=-45, max=45, step=9, value=25.0, description="$\chi$ (deg)")
psi_deg = FloatSlider(min=-90, max=90, step=9, value=45.0, description="$\psi$ (deg)")

# Stack sliders vertically
box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    justify_content='center',
                    border='none',
                    width='50%')

ui = VBox([r_val, chi_deg, psi_deg], layout=box_layout)

To make the phase of the complex-valued analytic representation of the electromagnetic match that of the real-valued wave described by Equation (2) of Stokes (1852), it is necessary to delay the analytic signal by $\pi/2$ radians of phase, which is equivalent to multiplying by $-j$.  Changing the absolute phase of the signal has no impact on polarization state and is not strictly necessary; however, in this simulation, matching the phases of the real-valued and complex-valued waves helps to predict where the arrows will be drawn.

To understand the need for the $-90$ degree phase shift, consider the primed coordinate system, where $\psi = 0$. Equation (2) of Stokes (1852) and Equation (4) of van Straten & Tiburzi (2025) describe the real-valued components of the electric field vector in this frame of reference,

${\bf\epsilon}^\prime(t) =
        \left(
        \begin{array}{c}
                \epsilon_x^\prime(t) \\
                \epsilon_y^\prime(t)
        \end{array}
        \right) =
        \left(
        \begin{array}{c}
                r \cos\chi \sin (2\pi \nu t) \\
                r \sin\chi \cos (2\pi \nu t)
        \end{array}
        \right),$
        
where, without any loss of generality, the absolute phase $\phi$ is set to zero. Similarly, based on Equation (5) of van Straten & Tiburzi (2025), the analytic signal associated with ${\bf\epsilon}^\prime(t)$ is given by

${\bf e}^\prime(t)=
  r\left( \begin{array}{c}
    \cos\chi \\
    j\sin\chi
  \end{array} \right) \exp (2\pi \nu t + \delta),$

where $\delta$ is the phase difference between the real-valued and complex-valued representations of the signal.  The real part,

$\mathcal{R}\left[{\bf e}^\prime(t)\right] =r\left( \begin{array}{c}
    \cos\chi\cos (2\pi \nu t + \delta)  \\
   -\sin\chi\sin (2\pi \nu t + \delta)
  \end{array} \right)$,
  
and setting ${\bf\epsilon}^\prime(t) = \mathcal{R}\left[{\bf e}^\prime(t)\right]$ yields $\delta = -\pi/2$.

In [2]:
# 60 steps around ellipse (+1 because 0 and 2pi overlap)
N = 61
phi = linspace(0, 2*pi, N)

# Arrays of cosines and sines
cosph = cos(phi)
sinph = sin(phi)

# Array of complex exponential values (-j * see note above)
expph = -1j * exp(1j*phi)

## Callback computes Equations (4) and (5) and sets data in plot objects
def plot_ellipse_comparison(r_val,chi_deg,psi_deg):
        
    r = r_val
    chi = chi_deg*pi/180.0
    psi = psi_deg*pi/180.0
        
    ## Equation 4
    xprime = r*cos(chi)*sinph
    yprime = r*sin(chi)*cosph
        
    ## Inverse of Equations 2 and 3
    x = xprime*cos(psi) - yprime*sin(psi)
    y = xprime*sin(psi) + yprime*cos(psi)
        
    real_ellipse.set_data(x, y)

    # draw real arrow at zero phase
    q=N-1
    real_arrow.set_data(x=x[q], y=y[q], dx=x[0]-x[q], dy=y[0]-y[q])

    ## Equation 5
    e_x = r * complex( cos(psi)*cos(chi), -sin(psi)*sin(chi) ) * expph
    e_y = r * complex( sin(psi)*cos(chi),  cos(psi)*sin(chi) ) * expph

    x = real(e_x)
    y = real(e_y)
        
    comp_ellipse.set_data(x, y)
    
    # draw complex arrow at 180 deg phase
    q=N//2-1
    comp_arrow.set_data(x=x[q], y=y[q], dx=x[q+1]-x[q], dy=y[q+1]-y[q])

    display(fig)

## Attach slider bars to plot_ellipse_comparison callback
w = interactive_output(plot_ellipse_comparison,{'r_val':r_val, 'chi_deg':chi_deg, 'psi_deg':psi_deg})
    
# Horizontal Layout puts sliders beside plot
display(HBox([w,ui]))